# Step 1: Build the lens catalog with images, and produce NPE mass models #

### Note: This notebook runs in the paltas_preds_env ###

This env is managed separately from the general forecast_env, since it relies on tensorflow

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py
import sys
sys.path.insert(0, '/Users/smericks/Desktop/StrongLensing/darkenergy-from-LAGN/')
import Modeling.MassModels.paltas_preds as paltas_preds

Load in pre-trained neural network modeler (one for HST quality, another for LSST quality)

In [ ]:
lsst_preds= paltas_preds.PaltasPreds('/Users/smericks/Desktop/StrongLensing/darkenergy-from-LAGN/Modeling/MassModels/lsst_training_config.py',
    numpix=33,
    model_weights='/Users/smericks/Desktop/StrongLensing/darkenergy-from-LAGN/Modeling/MassModels/_58--6.35_last.h5',#xresnet34_lsst_epoch76.h5',
    model_norms='/Users/smericks/Desktop/StrongLensing/darkenergy-from-LAGN/Modeling/MassModels/norms_for_sydney.csv',
    norm_images='norm')

hst_preds = paltas_preds.PaltasPreds('/Users/smericks/Desktop/StrongLensing/darkenergy-from-LAGN/Modeling/MassModels/hst_training_config.py',
    numpix=165,
    model_weights='/Users/smericks/Desktop/StrongLensing/darkenergy-from-LAGN/Modeling/MassModels/xresnet34_hst_epoch72.h5',
    model_norms='/Users/smericks/Desktop/StrongLensing/darkenergy-from-LAGN/Modeling/MassModels/hst_norms.csv')

Generate images, metadata, and NPE predictions

In [ ]:
# save images, preds, metadata
lenses = pd.read_csv('DataVectors/om10_venkatraman25_withkin.csv')
images_hst, metadata_list_hst, y_pred_hst, std_pred_hst, cov_pred_hst = hst_preds.preds_from_params(lenses)
images_lsst, metadata_list_lsst, y_pred_lsst, std_pred_lsst, cov_pred_lsst = lsst_preds.preds_from_params(lenses)

Construct metadata dataframes and add in pre-computed kinematics

In [ ]:
metadata_df_hst = pd.DataFrame(metadata_list_hst)
metadata_df_lsst = pd.DataFrame(metadata_list_lsst)

metadata_df_hst['sigma_v_4MOST_kmpersec'] = lenses.loc[:,'sigma_v_4MOST_kmpersec']
metadata_df_lsst['sigma_v_4MOST_kmpersec'] = lenses.loc[:,'sigma_v_4MOST_kmpersec']

Deal with inconsistencies in number of image positions founds during lens equation solving 

(NOTE: I set the lens equation solver parameters to be identical, and I still see this issue. There must be some stochasticity in the process? Maybe in the random starting points? )

In [ ]:
plt.hist(metadata_df_hst.loc[:,'point_source_parameters_num_images'])
plt.hist(metadata_df_lsst.loc[:,'point_source_parameters_num_images'])

hst_num_images = metadata_df_hst.loc[:,'point_source_parameters_num_images'].to_numpy()
lsst_num_images = metadata_df_lsst.loc[:,'point_source_parameters_num_images'].to_numpy()

# cut lenses that don't have same # of image positions in both rounds of image generation
indices_doubles = np.where((hst_num_images == 2) & (lsst_num_images == 2))[0]
indices_quads = np.where((hst_num_images == 4) & (lsst_num_images == 4))[0]

# final total indices
lens_idxs = np.hstack((indices_doubles,indices_quads))

# track a unique catalog_idx that will stay with the lens from now on...
metadata_df_hst['catalog_idx'] = np.arange(0,len(metadata_df_hst))
metadata_df_lsst['catalog_idx'] = np.arange(0,len(metadata_df_hst))

# some debugging stuff...
"""
print('num HST quads: %d'%(np.sum(metadata_df_hst.loc[:,'point_source_parameters_num_images'] == 4)))
print('num LSST quads: %d'%(np.sum(metadata_df_lsst.loc[:,'point_source_parameters_num_images'] == 4)))

print('num HST doubles: %d'%(np.sum(metadata_df_hst.loc[:,'point_source_parameters_num_images'] == 2)))
print('num LSST doubles: %d'%(np.sum(metadata_df_lsst.loc[:,'point_source_parameters_num_images'] == 2)))

# check that they're the same quads
indices_hst = metadata_df_hst.loc[metadata_df_hst['point_source_parameters_num_images'] == 2].index
indices_lsst = metadata_df_lsst.loc[metadata_df_lsst['point_source_parameters_num_images'] == 2].index
np.allclose(indices_hst,indices_lsst)
"""

Save information to .h5 files

In [ ]:
def save_h5(h5_path,catalog_idxs,images,mu_npe,cov_npe):
    h5f = h5py.File(h5_path, 'w')
    h5f.create_dataset('catalog_idx', data=catalog_idxs)
    h5f.create_dataset('images_array', data=images)
    h5f.create_dataset('mu_npe',data=mu_npe)
    h5f.create_dataset('cov_npe',data=cov_npe)
    h5f.close()


# GOLD
metadata_gold = metadata_df_hst.loc[lens_idxs]
metadata_gold.to_csv('DataVectors/gold/truth_metadata.csv', index=False)
save_h5('DataVectors/gold/image_models.h5',
        catalog_idxs=metadata_gold.loc[:,'catalog_idx'].to_numpy(),
        images=images_hst[lens_idxs],
        mu_npe=y_pred_hst[lens_idxs],
        cov_npe=cov_pred_hst[lens_idxs])

# SILVER
metadata_silver = metadata_df_lsst.loc[lens_idxs]
metadata_silver.to_csv('DataVectors/silver/truth_metadata.csv', index=False)
save_h5('DataVectors/silver/image_models.h5',
        catalog_idxs=metadata_silver.loc[:,'catalog_idx'].to_numpy(),
        images=images_lsst[lens_idxs],
        mu_npe=y_pred_lsst[lens_idxs],
        cov_npe=cov_pred_lsst[lens_idxs])


Sanity check the new silver-quality mass models

In [ ]:
truth_gamma = metadata_silver.loc[:,'main_deflector_parameters_gamma'].to_numpy()
pred_gamma = y_pred_lsst[lens_idxs,3]

plt.scatter(truth_gamma,pred_gamma)
x_low = 1.5
x_high = 2.5
plt.plot([x_low,x_high],[x_low,x_high],color='black')
plt.xlim([x_low,x_high])
plt.ylim([x_low,x_high])
plt.xlabel('Truth gamma')
plt.ylabel('Pred gamma')